In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from  sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import gc

In [2]:
NFOLDS = 3
SEED = 0
NROWS = None 

data = pd.read_csv('C:/Users/sln/Desktop/项目/home-credit-default-risk/application_train.csv')
test = pd.read_csv('C:/Users/sln/Desktop/项目/home-credit-default-risk/application_test.csv')
prev = pd.read_csv('C:/Users/sln/Desktop/项目/home-credit-default-risk/previous_application.csv')

In [3]:
categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]

for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])
    
gc.enable()

y_train = data['TARGET']
del data['TARGET']

In [4]:
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])
    
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

x_train = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
x_test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')

x_train = x_train.fillna(0)
x_test= x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

excluded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in excluded_feats]

x_train = x_train[features]
x_test = x_test[features]

kf = KFold(n_splits = NFOLDS, shuffle=True, random_state=SEED)

In [5]:


        
class LightGBMWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        if params is None:
            params = {}
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
    def _get_param_names(cls):
        """Get parameter names for the estimator"""
        # fetch the constructor or the original constructor before
        # deprecation wrapping if any
        init = getattr(cls.__init__, 'deprecated_original', cls.__init__)
        if init is object.__init__:
            # No explicit constructor to introspect
            return []

        # introspect the constructor arguments to find the model parameters
        # to represent
        init_signature = signature(init)
        # Consider the constructor parameters excluding 'self'
        parameters = [p for p in init_signature.parameters.values()
                      if p.name != 'self' and p.kind != p.VAR_KEYWORD]
        for p in parameters:
            if p.kind == p.VAR_POSITIONAL:
                raise RuntimeError("scikit-learn estimators should always "
                                   "specify their parameters in the signature"
                                   " of their __init__ (no varargs)."
                                   " %s with constructor %s doesn't "
                                   " follow this convention."
                                   % (cls, init_signature))
        # Extract and sort argument names excluding 'self'
        return sorted([p.name for p in parameters])

    def get_params(self, deep=True):
        out = dict()
        for key in self._get_param_names():
            # We need deprecation warnings to always be on in order to
            # catch deprecated param values.
            # This is set in utils/__init__.py but it gets overwritten
            # when running under python3 somehow.
            warnings.simplefilter("always", DeprecationWarning)
            try:
                with warnings.catch_warnings(record=True) as w:
                    value = getattr(self, key, None)
                if len(w) and w[0].category == DeprecationWarning:
                    # if the parameter is deprecated, don't show it
                    continue
            finally:
                warnings.filters.pop(0)

            # XXX: should we rather test if instance of estimator?
            if deep and hasattr(value, 'get_params'):
                deep_items = value.get_params().items()
                out.update((key + '__' + k, val) for k, val in deep_items)
            out[key] = value
        return out


class XgbWrapper(object):
    def __init__(self, seed=0, params=None):
        self.param = params
        if params is None:
            params = {}
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 250)

    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label=y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))
    
    def _get_param_names(cls):
        """Get parameter names for the estimator"""
        # fetch the constructor or the original constructor before
        # deprecation wrapping if any
        init = getattr(cls.__init__, 'deprecated_original', cls.__init__)
        if init is object.__init__:
            # No explicit constructor to introspect
            return []

        # introspect the constructor arguments to find the model parameters
        # to represent
        init_signature = signature(init)
        # Consider the constructor parameters excluding 'self'
        parameters = [p for p in init_signature.parameters.values()
                      if p.name != 'self' and p.kind != p.VAR_KEYWORD]
        for p in parameters:
            if p.kind == p.VAR_POSITIONAL:
                raise RuntimeError("scikit-learn estimators should always "
                                   "specify their parameters in the signature"
                                   " of their __init__ (no varargs)."
                                   " %s with constructor %s doesn't "
                                   " follow this convention."
                                   % (cls, init_signature))
        # Extract and sort argument names excluding 'self'
        return sorted([p.name for p in parameters])

    def get_params(self, deep=True):
        out = dict()
        for key in self._get_param_names():
            # We need deprecation warnings to always be on in order to
            # catch deprecated param values.
            # This is set in utils/__init__.py but it gets overwritten
            # when running under python3 somehow.
            warnings.simplefilter("always", DeprecationWarning)
            try:
                with warnings.catch_warnings(record=True) as w:
                    value = getattr(self, key, None)
                if len(w) and w[0].category == DeprecationWarning:
                    # if the parameter is deprecated, don't show it
                    continue
            finally:
                warnings.filters.pop(0)

            # XXX: should we rather test if instance of estimator?
            if deep and hasattr(value, 'get_params'):
                deep_items = value.get_params().items()
                out.update((key + '__' + k, val) for k, val in deep_items)
            out[key] = value
        return out


In [6]:
class CatboostWrapper(object):
    def __init__(self, clf, seed=0, params=None):
        if params is None:
            params = {}
        params['random_seed'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)[:,1]
    def _get_param_names(cls):
        """Get parameter names for the estimator"""
        # fetch the constructor or the original constructor before
        # deprecation wrapping if any
        init = getattr(cls.__init__, 'deprecated_original', cls.__init__)
        if init is object.__init__:
            # No explicit constructor to introspect
            return []

        # introspect the constructor arguments to find the model parameters
        # to represent
        init_signature = signature(init)
        # Consider the constructor parameters excluding 'self'
        parameters = [p for p in init_signature.parameters.values()
                      if p.name != 'self' and p.kind != p.VAR_KEYWORD]
        for p in parameters:
            if p.kind == p.VAR_POSITIONAL:
                raise RuntimeError("scikit-learn estimators should always "
                                   "specify their parameters in the signature"
                                   " of their __init__ (no varargs)."
                                   " %s with constructor %s doesn't "
                                   " follow this convention."
                                   % (cls, init_signature))
        # Extract and sort argument names excluding 'self'
        return sorted([p.name for p in parameters])

    def get_params(self, deep=True):
        out = dict()
        for key in self._get_param_names():
            # We need deprecation warnings to always be on in order to
            # catch deprecated param values.
            # This is set in utils/__init__.py but it gets overwritten
            # when running under python3 somehow.
            warnings.simplefilter("always", DeprecationWarning)
            try:
                with warnings.catch_warnings(record=True) as w:
                    value = getattr(self, key, None)
                if len(w) and w[0].category == DeprecationWarning:
                    # if the parameter is deprecated, don't show it
                    continue
            finally:
                warnings.filters.pop(0)

            # XXX: should we rather test if instance of estimator?
            if deep and hasattr(value, 'get_params'):
                deep_items = value.get_params().items()
                out.update((key + '__' + k, val) for k, val in deep_items)
            out[key] = value
        return out

In [7]:
def get_oof(clf):
    #np.zeros()返回来一个给定形状和类型的用0填充的数组；
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    #np.empty()返回随机数组
    oof_test_skf = np.empty((NFOLDS, ntest))
#enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [8]:
et_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

rf_params = {
    'n_jobs': 16,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

In [9]:
lightgbm_params = {
    'n_estimators':200,
    'learning_rate':0.1,
    'num_leaves':123,
    'colsample_bytree':0.8,
    'subsample':0.9,
    'max_depth':15,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'min_split_gain':0.01,
    'min_child_weight':2    
}

In [10]:
catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

In [11]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
#et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
#rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf= CatBoostClassifier, seed = SEED, params=catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

xg_oof_train, xg_oof_test = get_oof(xg)
#et_oof_train, et_oof_test = get_oof(et)
lg_oof_train, lg_oof_test = get_oof(lg)
cb_oof_train, cb_oof_test = get_oof(cb)

[11:13:51] WARNING: D:\Build\xgboost\xgboost-1.1.0.git\python-package\build\temp.win-amd64-3.7\Release\xgboost\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:14:31] WARNING: D:\Build\xgboost\xgboost-1.1.0.git\python-package\build\temp.win-amd64-3.7\Release\xgboost\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:15:12] WARNING: D:\Build\xgboost\xgboost-1.1.0.git\python-package\build\temp.win-amd64-3.7\Release\xgboost\src\learner.cc:480: 
Parameters: { silent } might not be used

4:	total: 327ms	remaining: 12.8s
5:	total: 390ms	remaining: 12.6s
6:	total: 447ms	remaining: 12.3s
7:	total: 507ms	remaining: 12.2s
8:	total: 567ms	remaining: 12s
9:	total: 624ms	remaining: 11.9s
10:	total: 683ms	remaining: 11.7s
11:	total: 741ms	remaining: 11.6s
12:	total: 804ms	remaining: 11.6s
13:	total: 869ms	remaining: 11.6s
14:	total: 932ms	remaining: 11.5s
15:	total: 993ms	remaining: 11.4s
16:	total: 1.05s	remaining: 11.3s
17:	total: 1.12s	remaining: 11.3s
18:	total: 1.18s	remaining: 11.2s
19:	total: 1.23s	remaining: 11.1s
20:	total: 1.31s	remaining: 11.1s
21:	total: 1.4s	remaining: 11.3s
22:	total: 1.49s	remaining: 11.5s
23:	total: 1.57s	remaining: 11.5s
24:	total: 1.64s	remaining: 11.5s
25:	total: 1.71s	remaining: 11.4s
26:	total: 1.76s	remaining: 11.3s
27:	total: 1.82s	remaining: 11.2s
28:	total: 1.88s	remaining: 11.1s
29:	total: 1.95s	remaining: 11s
30:	total: 2s	remaining: 10.9s
31:	total: 2.06s	remaining: 10.8s
32:	total: 2.13s	remaining: 10.8s
33:	total: 2.2s	remaining: 1

47:	total: 3.04s	remaining: 9.63s
48:	total: 3.11s	remaining: 9.58s
49:	total: 3.17s	remaining: 9.51s
50:	total: 3.23s	remaining: 9.44s
51:	total: 3.3s	remaining: 9.39s
52:	total: 3.37s	remaining: 9.34s
53:	total: 3.43s	remaining: 9.27s
54:	total: 3.49s	remaining: 9.21s
55:	total: 3.56s	remaining: 9.15s
56:	total: 3.62s	remaining: 9.07s
57:	total: 3.68s	remaining: 9.01s
58:	total: 3.74s	remaining: 8.94s
59:	total: 3.81s	remaining: 8.88s
60:	total: 3.86s	remaining: 8.8s
61:	total: 3.92s	remaining: 8.73s
62:	total: 3.98s	remaining: 8.65s
63:	total: 4.05s	remaining: 8.6s
64:	total: 4.1s	remaining: 8.52s
65:	total: 4.17s	remaining: 8.47s
66:	total: 4.23s	remaining: 8.39s
67:	total: 4.29s	remaining: 8.33s
68:	total: 4.35s	remaining: 8.25s
69:	total: 4.39s	remaining: 8.16s
70:	total: 4.46s	remaining: 8.11s
71:	total: 4.53s	remaining: 8.05s
72:	total: 4.59s	remaining: 7.99s
73:	total: 4.65s	remaining: 7.92s
74:	total: 4.73s	remaining: 7.88s
75:	total: 4.78s	remaining: 7.8s
76:	total: 4.84s	re

In [12]:
print("XG-CV: {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("LG-CV: {}".format(sqrt(mean_squared_error(y_train, lg_oof_train))))
print("CB-CV: {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))



XG-CV: 0.25969546149690725
LG-CV: 0.26082255425773715
CB-CV: 0.3309876006961808


In [13]:
x_train = np.concatenate((xg_oof_train, lg_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, lg_oof_test,  cb_oof_test), axis=1)


In [14]:

print("{},{}".format(x_train.shape, x_test.shape))
#deepFM
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train) 

#test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]
test['TARGET'] = logistic_regression.predict_proba(x_test)[:,1]
test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index=False, float_format='%.8f')

(307511, 3),(48744, 3)


In [15]:
from sklearn.ensemble import VotingClassifier 

In [16]:
#from sklearn.utils.fixes import signature
from inspect import signature

In [17]:
import warnings

In [18]:
voting_clf = VotingClassifier(estimators=[('xgboost',xg),('lightgbm',lg),('catboost',cb)],#estimators:子分类器
                              voting='soft') #参数voting代表你的投票方式，hard,soft
# voting_clf.fit(X_train,y_train)
xg._estimator_type = "classifier"
lg._estimator_type = "classifier"
cb._estimator_type = "classifier"
voting_clf.fit(x_train,y_train)
votingg_pred = voting_clf.predict(x_test)

TypeError: 'NoneType' object does not support item assignment